# Лекция 4: Аппроксимация данных

## Аппроксимация

### Постановка задачи
Задача аппроксимации формулируется следующим образом: дано множество обучающих данных  
$$
\{(x_i, y_i)\}_{i=1}^{N}, \quad x_i \in \mathbb{R}^d, \; y_i \in \mathbb{R},
$$
требуется найти функцию $ f: \mathbb{R}^d \to \mathbb{R} $, которая удовлетворяет условию:
$$
y_i \approx f(x_i), \quad \forall i = 1, \dots, N.
$$

Обычно задача решается через минимизацию функции потерь, например, суммы квадратов ошибок:
$$
\min_{f \in \mathcal{F}} \; \sum_{i=1}^{N} \left( y_i - f(x_i) \right)^2,
$$
где $\mathcal{F}$ — множество рассматриваемых моделей. Такой подход позволяет не только "подогнать" модель под данные, но и контролировать обобщающую способность модели через понятия смещения (bias) и дисперсии (variance).

#### Классические методы

Например, можно построить интерполяционный полином Лагранжа для набора точек $\{(x_i, y_i)\}_{i=1}^{N}$ определяется как:
$$
L(x) = \sum_{i=0}^{n} y_i \cdot l_i(x)
$$

где базисные полиномы $ l_i(x) $ определяются как:
$$
l_i(x) = \prod_{\substack{0 \le j \le n \\ j \neq i}} \frac{x - x_j}{x_i - x_j}
$$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


def f(x: float):
    return np.sin(x) * x/5

def lagrange_interpolation(x_points, 
                           y_points, 
                           x):
    """
    Интерполяция Лагранжа

    Аргументы:
        x_points (list[float]): Список значений x
        y_points (list[float]): Список значений y
        x (float): Значение x для интерполяции

    Возвращает:
        float: Значение интерполяции
    """
    def basis_polynomial(i, x):
        """
        Базисный полином Лагранжа

        Аргументы:
            i (int): Индекс базисного полинома
            x (float): Значение x для интерполяции

        Возвращает:
            float: Значение базисного полинома
        """
        terms = [
            (x - x_points[j]) / (x_points[i] - x_points[j])
            for j in range(len(x_points)) if j != i
        ]
        return np.prod(terms, axis=0)

    return (sum(y_points[i] * basis_polynomial(i, x) 
                for i in range(len(x_points))))

N = 10
x_points = np.linspace(0, 5, N)
y_points = f(x_points)

x_new = np.linspace(0, 5, 100)
y_lagrange = lagrange_interpolation(x_points, 
                                    y_points, 
                                    x_new)

plt.figure(figsize=(6, 6))
plt.plot(x_points, y_points, 'bo', label='Исходные точки')
plt.plot(x_new, y_lagrange, 'r-', label='Интерполяция Лагранжа')
plt.plot(x_new, f(x_new), 'g--', label='Оригинальная функция')
plt.xlabel('x')
plt.ylabel('y')
plt.title('Интерполяция Лагранжа')
plt.axis('equal')
plt.legend()
plt.show()

### Универсальная аппроксимационная теорема

Универсальная аппроксимационная теорема утверждает следующее:

Пусть $ f: \mathbb{R}^n \to \mathbb{R} $ — непрерывная функция, определённая на компактном подмножестве $\mathbb{R}^n$. Тогда для любого $\epsilon > 0$ существует нейронная сеть с одним скрытым слоем, использующая нелинейную активационную функцию $\sigma$, такая, что:

$$
\left| f(x) - \sum_{i=1}^{N} a_i \sigma(\langle w_i, x \rangle + b_i) \right| < \epsilon
$$

для всех $ x $ из этого компактного подмножества, где:
- $ N $ — количество нейронов в скрытом слое,
- $ a_i, w_i, b_i $ — параметры сети (веса и смещения),
- $\sigma$ — нелинейная активационная функция, например, сигмоидальная функция.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt


class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.linear1 = nn.Linear(1, 16)
        self.linear2 = nn.Linear(16, 1)

    def forward(self, x):
        x = torch.tanh(self.linear1(x))
        x = self.linear2(x)
        return x


N = 10
x_points = np.linspace(0, 5, N)
y_points = f(x_points)

x_train = torch.tensor(x_points, 
                       dtype=torch.float32).view(-1, 1)
y_train = torch.tensor(y_points, 
                       dtype=torch.float32).view(-1, 1)

model = SimpleNN()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

epochs = 10000
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    output = model(x_train)
    loss = criterion(output, y_train)
    loss.backward()
    optimizer.step()

x_test = torch.tensor(x_new, dtype=torch.float32).view(-1, 1)
y_pred_nn = model(x_test).detach().numpy()

mse_lagrange = np.mean((y_lagrange - f(x_new))**2)
mse_nn = np.mean((y_pred_nn.flatten() - f(x_new))**2)

print(f"MSE Лагранжа: {mse_lagrange:.4f}")
print(f"MSE Нейронной сети: {mse_nn:.4f}")

plt.figure(figsize=(6, 6))
plt.plot(x_points, y_points, 'bo', label='Исходные точки')
plt.plot(x_test.numpy(), y_pred_nn, 'r-', label='Нейронная сеть')
plt.plot(x_test.numpy(), np.sin(x_test.numpy()) * x_test.numpy() / 5, 
         'g--', label='Оригинальная функция sin(x)')

plt.xlabel('x')
plt.ylabel('y')
plt.title('Сравнение методов аппроксимации')
plt.axis('equal')
plt.legend()
plt.show()

### Линейные и нелинейные методы аппроксимации

#### Линейные методы
В линейном случае предполагается, что зависимость между $ x $ и $ y $ может быть описана линейным соотношением:
$$
f(x) = \langle \mathbf{w}, x \rangle + b,
$$
где:
- $\mathbf{w} \in \mathbb{R}^d$ — вектор коэффициентов,
- $b \in \mathbb{R}$ — смещение.

Обучение модели (например, с использованием метода наименьших квадратов) сводится к решению задачи:
$$
\min_{\mathbf{w}, \, b} \; \sum_{i=1}^{N} \left( y_i - (\langle \mathbf{w}, x_i \rangle + b) \right)^2.
$$
Преимущества линейных моделей:
- Простота и хорошая интерпретируемость;
- Низкая вычислительная сложность.

Ограничение заключается в том, что они не способны адекватно аппроксимировать сложные нелинейные зависимости.

#### Нелинейные методы
Чтобы учитывать нелинейности, можно использовать преобразование исходных признаков. Одним из подходов является введение нового отображения $\phi: \mathbb{R}^d \to \mathbb{R}^{d'}$, приводящего к модели:
$$
f(x) = \langle \mathbf{w}, \phi(x) \rangle + b.
$$

Примером является полиномиальная регрессия, где функция $\phi(x)$ включает полиномы входных признаков:
$$
\phi(x) = [1, x, x^2, \dots, x^p],
$$
и модель становится:
$$
f(x) = w_0 + w_1 x + w_2 x^2 + \dots + w_p x^p.
$$
Преимущества нелинейных методов:
- Гибкость в аппроксимации сложных зависимостей;
- Возможность выбора степени нелинейности через параметр $p$.

Недостатки:
- Рост числа параметров может привести к переобучению;
- Снижение интерпретируемости модели.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn


class SingleNeuron(nn.Module):
    def __init__(self, activation):
        super(SingleNeuron, self).__init__()
        self.linear = nn.Linear(1, 1)
        self.activation = activation

    def forward(self, x):
        x = self.linear(x)
        x = self.activation(x)
        return x


x_points = np.linspace(-10, 10, 100, dtype=np.float32).reshape(-1, 1)

activations = {
    'ReLU': torch.relu,
    'Sigmoid': torch.sigmoid,
    'Tanh': torch.tanh
}

fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(15, 4))

for ax, (name, activation) in zip(axes, activations.items()):
    model = SingleNeuron(activation)
    with torch.no_grad():
        y_pred = model(torch.tensor(x_points)).numpy()
    
    ax.plot(x_points, y_pred, label=f'Активация: {name}')
    ax.set_title(f'Функция активации: {name}')
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.legend()

plt.tight_layout()
plt.show()

In [ ]:
model = SingleNeuron(torch.relu)

for name, param in model.named_parameters():
    print(f"{name}, {param.data}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim


class SimpleNN(nn.Module):
    def __init__(self, activation, num_neurons):
        super(SimpleNN, self).__init__()
        self.linear1 = nn.Linear(1, num_neurons)
        self.activation = activation
        self.linear2 = nn.Linear(num_neurons, 1)

    def forward(self, x):
        x = self.activation(self.linear1(x))
        x = self.linear2(x)
        return x


def f(x):
    return np.sin(x) * x / 5


x_points = np.linspace(0, 5, 100, dtype=np.float32)
y_points = f(x_points)

x_train = torch.tensor(x_points, dtype=torch.float32).view(-1, 1)
y_train = torch.tensor(y_points, dtype=torch.float32).view(-1, 1)


def train_and_plot(activation, num_neurons, ax, title):
    model = SimpleNN(activation, num_neurons)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    epochs = 5000
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        output = model(x_train)
        loss = criterion(output, y_train)
        loss.backward()
        optimizer.step()

    y_pred = model(x_train).detach().numpy()

    ax.plot(x_points, y_points, 'g--', label='Оригинальная функция')
    ax.plot(x_points, y_pred, 'r-', label='Аппроксимация')
    ax.set_title(title)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.legend()

activations = {
    'ReLU': torch.relu,
    'Sigmoid': torch.sigmoid,
    'Tanh': torch.tanh
}

num_neurons_list = [1, 5, 10, 50]
fig, axes = plt.subplots(nrows=len(num_neurons_list), 
                         ncols=3, figsize=(15, 12))

for col, (name, activation) in enumerate(activations.items()):
    for row, num_neurons in enumerate(num_neurons_list):
        train_and_plot(activation, num_neurons, axes[row, col], 
                       f'{name} с {num_neurons} нейронами')

plt.tight_layout()
plt.show()

### Ядерные методы и метод опорных векторов

#### Ядерные методы
Ядерные методы опираются на идею явного или неявного переноса данных в пространство более высокой размерности, где зависимость становится линейной. Пусть имеется отображение $\phi: \mathbb{R}^d \to \mathcal{H}$, тогда модель выглядит как:
$$
f(x) = \langle \mathbf{w}, \phi(x) \rangle + b.
$$
При этом скалярное произведение в пространстве $\mathcal{H}$ вычисляется с помощью функции ядра $K(x, z)$ по правилу:
$$
K(x, z) = \langle \phi(x), \phi(z) \rangle.
$$

#### Основные типы ядер

1. **Линейное ядро**:
   $$
   K(x, z) = x^T z
   $$
   - Используется, когда данные линейно разделимы.

2. **Полиномиальное ядро**:
   $$
   K(x, z) = (\gamma x^T z + r)^d
   $$
   - $\gamma$ — коэффициент, $r$ — смещение, $d$ — степень полинома.
   - Позволяет моделировать полиномиальные зависимости.

3. **Радиальная базисная функция (RBF) или Гауссово ядро**:
   $$
   K(x, z) = \exp\left(-\frac{\|x - z\|^2}{2\sigma^2}\right)
   $$
   - $\sigma$ — параметр, определяющий ширину гауссиана.
   - Хорошо подходит для нелинейно разделимых данных.

4. **Сигмоидное ядро**:
   $$
   K(x, z) = \tanh(\gamma x^T z + r)
   $$
   - Похоже на активационную функцию нейронных сетей.


#### Метод опорных векторов (SVM)

[Подробно](http://www.machinelearning.ru/wiki/images/archive/4/43/20150402092440!Voron-ML-regress-non-slides.pdf)

#### Основная идея
Для линейно разделимых данных SVM ищет гиперплоскость, которая разделяет данные с максимальным зазором. Гиперплоскость в $ n $-мерном пространстве определяется уравнением:
$$
\mathbf{w} \cdot \mathbf{x} + b = 0
$$

где:
- $\mathbf{w}$ — вектор весов,
- $b$ — смещение,
- $\mathbf{x}$ — вектор признаков.

#### Максимизация зазора
SVM стремится максимизировать зазор между двумя классами. Границы зазора определяются уравнениями:
$$
\mathbf{w} \cdot \mathbf{x} + b = 1
$$
$$
\mathbf{w} \cdot \mathbf{x} + b = -1
$$

Расстояние между этими двумя гиперплоскостями равно $ \frac{2}{\|\mathbf{w}\|} $. Таким образом, задача SVM сводится к минимизации $\|\mathbf{w}\|$ при условии, что все точки классифицируются правильно:

$$
y_i (\mathbf{w} \cdot \mathbf{x}_i + b) \geq 1, \quad \forall i
$$

где $ y_i $ — метка класса для точки $\mathbf{x}_i$.


#### Отступ (margin)
Определяется как:
$$
M = y_i (\mathbf{w} \cdot \mathbf{x}_i)
$$

Положительный отступ указывает на правильную классификацию, отрицательный — на ошибку.


#### Оптимизация
Для минимизации ошибок классификации используются различные функции потерь, такие как Hinge Loss:
$$
L(w, x, y) = \lambda \|\mathbf{w}\|^2 + \sum_{i} \max(0, 1 - y_i (\mathbf{w} \cdot \mathbf{x}_i))
$$

#### Ядровый трюк

Для нелинейно разделимых данных SVM использует ядровую функцию $ K(\mathbf{x}_i, \mathbf{x}_j) $, чтобы неявно преобразовать данные в более высокоразмерное пространство, где они могут быть линейно разделимы. Ядровая функция заменяет скалярное произведение $\mathbf{x}_i \cdot \mathbf{x}_j$ в двойственной задаче.

Примеры ядер:
- Линейное: $ K(\mathbf{x}, \mathbf{z}) = \mathbf{x}^T \mathbf{z} $
- Полиномиальное: $ K(\mathbf{x}, \mathbf{z}) = (\gamma \mathbf{x}^T \mathbf{z} + r)^d $
- RBF (Гауссово): $ K(\mathbf{x}, \mathbf{z}) = \exp\left(-\frac{\|\mathbf{x} - \mathbf{z}\|^2}{2\sigma^2}\right) $


In [ ]:
import numpy as np
import matplotlib.pyplot as plt


M = np.linspace(-2, 2, 400)

def error(M):
    return np.maximum(0, np.sign(-M))

def hinge_loss(M):
    return np.maximum(0, 1 - M)

def perceptron_loss(M):
    return np.maximum(0, -M)

def squared_hinge_loss(M):
    return np.maximum(0, 1 - M)**2

plt.figure(figsize=(8, 8))
plt.plot(M, error(M), label='Error')
plt.plot(M, hinge_loss(M), label='Hinge Loss')
plt.plot(M, perceptron_loss(M), label='Perceptron Loss')
plt.plot(M, squared_hinge_loss(M), label='Squared Hinge Loss')
plt.axhline(0, color='black', linestyle='--')
plt.axvline(0, color='black', linestyle='--')
plt.title('Функции потерь')
plt.xlabel('Отступ (M)')
plt.ylabel('Значение функции потерь')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.datasets import make_circles


X, y = make_circles(n_samples=100, factor=0.3, noise=0.1, random_state=42)

kernels = ['linear', 'poly', 'rbf', 'sigmoid']

fig, axes = plt.subplots(2, 2, figsize=(12, 10))

for ax, kernel in zip(axes.ravel(), kernels):
    clf = svm.SVC(kernel=kernel, gamma='auto')
    clf.fit(X, y)

    xx, yy = np.meshgrid(np.linspace(-1.5, 1.5, 100), 
                         np.linspace(-1.5, 1.5, 100))
    
    Z = clf.decision_function(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    ax.contourf(xx, yy, Z, levels=np.linspace(Z.min(), 0, 7), cmap=plt.cm.PuBu)
    ax.contour(xx, yy, Z, levels=[0], linewidths=2, colors='darkred')
    ax.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.Paired, edgecolors='k')
    ax.set_title(f'Ядро: {kernel}')
    ax.set_xlim(-1.5, 1.5)
    ax.set_ylim(-1.5, 1.5)

plt.tight_layout()
plt.show()

#### Метод опорных векторов (SVR)
Метод опорных векторов для задачи регрессии, называемый Support Vector Regression (SVR), решает следующую задачу оптимизации:
$$
\begin{aligned}
&\min_{w,b,\xi_i,\xi_i^*}\quad \frac{1}{2} \|w\|^2 + C \sum_{i=1}^{N} (\xi_i + \xi_i^*) \\
&\text{при условиях:} \\
& y_i - \langle w, \phi(x_i) \rangle - b \le \epsilon + \xi_i, \\
& \langle w, \phi(x_i) \rangle + b - y_i \le \epsilon + \xi_i^*, \\
& \xi_i, \xi_i^* \ge 0, \quad i = 1,\dots,N.
\end{aligned}
$$
Здесь:
- $C$ — параметр, регулирующий штраф за ошибки,
- $\epsilon$ задаёт допустимый уровень неточности,
- $\xi_i, \xi_i^*$ — переменные, вводимые для учета нарушений допустимого отклонения.

В SVR цель — минимизировать ошибку предсказания, сохраняя простоту модели. Это достигается путем введения $\epsilon$ полосы (трубы) вокруг функции, в пределах которой ошибки не учитываются.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVR


np.random.seed(42)
X = np.sort(5 * np.random.rand(100, 1), axis=0)
y = np.sin(X).ravel()

epsilon = 0.1
svr_rbf = SVR(kernel='rbf', C=100, gamma=0.1, epsilon=epsilon)
svr_poly = SVR(kernel='poly', C=100, degree=3, epsilon=epsilon)
svr_linear = SVR(kernel='linear', C=100, epsilon=epsilon)

y_rbf = svr_rbf.fit(X, y).predict(X)
y_poly = svr_poly.fit(X, y).predict(X)
y_linear = svr_linear.fit(X, y).predict(X)

plt.figure(figsize=(6, 8))
plt.scatter(X, y, color='darkorange', 
            label='Данные', edgecolors='k')

plt.plot(X, y_rbf, color='navy', lw=2, label='RBF модель')
plt.fill_between(X.ravel(), 
                 y_rbf - epsilon, 
                 y_rbf + epsilon, 
                 color='navy', alpha=0.2)

plt.plot(X, y_poly, color='c', lw=2, label='Полиномиальная модель')
plt.fill_between(X.ravel(), 
                 y_poly - epsilon,
                 y_poly + epsilon, 
                 color='c', alpha=0.2)

plt.plot(X, y_linear, color='cornflowerblue', lw=2, label='Линейная модель')
plt.fill_between(X.ravel(), 
                 y_linear - epsilon, 
                 y_linear + epsilon, 
                 color='cornflowerblue', alpha=0.2)

plt.xlabel('x')
plt.ylabel('y')
plt.title('Support Vector Regression')
plt.legend()
plt.show()

### Гауссовские процессы

Гауссовские процессы (Gaussian Processes, GP) относятся к непараметрическим байесовским методам аппроксимации. Идея заключается в том, что функция $ f(x) $ рассматривается как случайный процесс, при котором любые конечные наборы значений функции имеют совместное мультиномальное нормальное распределение:
$$
f(x) \sim \mathcal{GP}\big(m(x),\, k(x, x') \big),
$$
где:
- $m(x)$ — функция среднего (обычно принимается равной нулю: $m(x)=0$),
- $k(x, x')$ — ковариационная функция (ядро), задающая степень схожести между точками $x$ и $x'$.

Одним из популярных ядер является **радиальная базисная функция (RBF)**, или Гауссово ядро:
$$
k(x, x') = \sigma_f^2 \exp\Big(-\frac{\|x - x'\|^2}{2l^2}\Big),
$$
где:
- $\sigma_f^2$ — дисперсия входных данных,
- $l$ — длина масштабирования.

**Преимущества Гауссовских процессов:**
- Непараметрический характер позволяет гибко адаптироваться к данным.
- Возможность получения не только точечного предсказания, но и оценки неопределенности (дисперсии) регрессионной функции.

**Ограничения:**
- Вычислительная сложность $ \mathcal{O}(N^3) $ при обучении, что затрудняет применение для больших наборов данных.
- Необходимость выбора ядра и его гиперпараметров, что может существенно влиять на качество аппроксимации.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C


def f(x):
    return np.sin(x) + 0.1 * np.random.randn(*x.shape)

X_train = np.array([[1], [3], [5], [6], [8]])
y_train = f(X_train)

kernel = (C(1.0, (1e-3, 1e3)) 
          * RBF(length_scale=1.0, 
                length_scale_bounds=(1e-2, 1e2)))

gp = GaussianProcessRegressor(kernel=kernel, 
                              n_restarts_optimizer=10)

gp.fit(X_train, y_train)

X_test = np.linspace(0, 10, 100).reshape(-1, 1)

y_pred, sigma = gp.predict(X_test, return_std=True)

plt.figure(figsize=(10, 5))
plt.plot(X_test, y_pred, 'r-', label='Предсказание')
plt.fill_between(X_test.ravel(), 
                 y_pred - 1.96 * sigma, 
                 y_pred + 1.96 * sigma, 
                 color='lightblue', alpha=0.5, label='Доверительный интервал')
plt.scatter(X_train, y_train, color='darkorange', label='Обучающие данные')
plt.title('Гауссовский процесс для регрессии')
plt.xlabel('x')
plt.ylabel('f(x)')
plt.legend()
plt.show()

[Интерактивная визуализация](https://distill.pub/2019/visual-exploration-gaussian-processes/)

[Интерактивная визуализация](https://www.infinitecuriosity.org/vizgp/)